#### Reading input data

In [1]:
input_path = "../_Staging/Pega_Bouncebacks_UserUnknow_012616.csv"

In [2]:
import datetime
import pandas as pd
from zoomtools import rdb

In [4]:
df = pd.read_csv(input_path)
df.dropna(subset=['Recipient Email Address'], inplace=True)
df.ix[:5, :5]

,DATE_CREATED,FILE_SOURCE,Recipient Email Address,Zoom Individual ID,VALIDATION
0,1/7/2016,ZoomInfo-2016,robert.s.smith@wellsfargo.com,-2102933875,Good-Catch-All
1,1/7/2016,ZoomInfo-2016,bellj@wellsfargo.com,1909134101,Good-Catch-All
2,12/11/2015,ZoomInfo-2015 Corporate - DEC,gary.methvin@airgas.com,1784688781,Good-Catch-All
3,12/11/2015,ZoomInfo-2015 Corporate - DEC,ronald.keller@airgas.com,-1986587797,Good-Catch-All
4,12/11/2015,ZoomInfo-2015 Corporate - DEC,herman.rios@airgas.com,2107267423,Good-Catch-All
5,12/11/2015,ZoomInfo-2015 Corporate - DEC,jesse.hemingway@airgas.com,-2019796696,Good-Catch-All


#### Creating database connection

In [5]:
rdb.drivers()

{0: 'SQL Server',
 1: 'Microsoft Access Driver (*.mdb, *.accdb)',
 2: 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 3: 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)',
 4: 'Microsoft Access Text Driver (*.txt, *.csv)',
 5: 'SQL Server Native Client 10.0',
 6: 'MySQL ODBC 5.2 ANSI Driver',
 7: 'MySQL ODBC 5.2 Unicode Driver',
 8: 'SQL Server Native Client 11.0',
 9: 'ODBC Driver 11 for SQL Server'}

In [6]:
ppdb = rdb.conn(driver=rdb.drivers()[8],
                server='Freshdb03',
                database='permission_pass_db',
                uid='sa')

Password for user sa:
········
Connected to permission_pass_db on Freshdb03.


#### Querying context data

In [7]:
def formatEmails(emails):
    escaped_emails = [email.replace("'", "''") for email in emails]
    email_string = ", ".join(["'" + email + "'" for email in escaped_emails])
    return email_string

In [8]:
query = """select email, date_validated, date_permission_passed
           from emails
           where email in ({})"""

In [10]:
email_string = formatEmails(df['Recipient Email Address'])
context = rdb.query(ppdb, query.format(email_string))
context.head()

,email,date_validated,date_permission_passed
0,alan.clark@transportscotland.gsi.gov.uk,2015-10-27,2012-01-07
1,aoconnor@manh.com,2015-12-04,2013-03-15
2,barb_faucette@cblproperties.com,2015-12-06,2012-01-18
3,barry.bearg@revlon.com,2015-10-31,2012-01-28
4,beth.horowitz@harris.com,2015-10-30,2012-01-06


#### Joining and saving to disk

In [12]:
master = df.merge(context, how='left', left_on='Recipient Email Address', right_on='email')
master.head()

,DATE_CREATED,FILE_SOURCE,Recipient Email Address,Zoom Individual ID,VALIDATION,Bounceback Date/Time,DATE,DOMAIN,SMTP Error Code,SMTP Reply Code,Bounceback Message,Email Name,email,date_validated,date_permission_passed
0,1/7/2016,ZoomInfo-2016,robert.s.smith@wellsfargo.com,-2102933875,Good-Catch-All,1/8/2016,1/8/2016,wellsfargo.com,5.1.1,NaN,#550 5.1.1 resolver.adr.recipnotfound; not fou...,NONE_PROVIDED,robert.s.smith@wellsfargo.com,2015-10-24,2015-04-20
1,1/7/2016,ZoomInfo-2016,bellj@wellsfargo.com,1909134101,Good-Catch-All,1/8/2016,1/8/2016,wellsfargo.com,5.1.1,NaN,#550 5.1.1 resolver.adr.recipnotfound; not fou...,NONE_PROVIDED,bellj@wellsfargo.com,2015-10-19,2014-09-30
2,12/11/2015,ZoomInfo-2015 Corporate - DEC,gary.methvin@airgas.com,1784688781,Good-Catch-All,1/8/2016,1/8/2016,airgas.com,5.1.1,NaN,#550 5.1.1 resolver.adr.recipnotfound; not fou...,NONE_PROVIDED,gary.methvin@airgas.com,2015-11-28,2015-06-16
3,12/11/2015,ZoomInfo-2015 Corporate - DEC,ronald.keller@airgas.com,-1986587797,Good-Catch-All,1/13/2016,1/13/2016,airgas.com,5.1.1,NaN,#550 5.1.1 resolver.adr.recipnotfound; not fou...,NONE_PROVIDED,ronald.keller@airgas.com,2015-10-31,2014-09-23
4,12/11/2015,ZoomInfo-2015 Corporate - DEC,herman.rios@airgas.com,2107267423,Good-Catch-All,1/13/2016,1/13/2016,airgas.com,5.1.1,NaN,#550 5.1.1 resolver.adr.recipnotfound; not fou...,NONE_PROVIDED,herman.rios@airgas.com,2015-11-28,2014-02-20


In [15]:
master.to_csv("Pega_Bouncebacks_UserUnknow_012616_dates_joined.csv", index=False)